In [62]:
#Set up the notebook environment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy
from scipy.stats import pearsonr
from scipy import signal as sig
import xgboost as xgb
from utils import *
from xgboost import XGBRegressor

In [72]:
raw = scipy.io.loadmat('./datasets/leaderboard_data.mat')
ecog_1 = raw['leaderboard_ecog'][0][0]
ecog_2 = raw['leaderboard_ecog'][1][0]
ecog_3 = raw['leaderboard_ecog'][2][0]

In [73]:
raw = scipy.io.loadmat('./datasets/raw_training_data.mat')
data_glove_1 = raw['train_dg'][0][0]
data_glove_1_train = np.delete(data_glove_1, 3, 1)
data_glove_2 = raw['train_dg'][1][0]
data_glove_2_train = np.delete(data_glove_2, 3, 1)
data_glove_3 = raw['train_dg'][2][0]
data_glove_3_train = np.delete(data_glove_3, 3, 1)

ecog_1_train = raw['train_ecog'][0][0]
ecog_2_train = raw['train_ecog'][1][0]
ecog_3_train = raw['train_ecog'][2][0]


raw = scipy.io.loadmat('./datasets/sub1_comp.mat')
ecog_1_comp = raw['train_data']
dg_1_comp = raw['train_dg']
ecog_1_valid = raw['test_data'][50000:200000]

raw = scipy.io.loadmat('./datasets/sub2_comp.mat')
ecog_2_comp = raw['train_data']
dg_2_comp = raw['train_dg']
ecog_2_valid = raw['test_data'][50000:200000]

raw = scipy.io.loadmat('./datasets/sub3_comp.mat')
ecog_3_comp = raw['train_data']
dg_3_comp = raw['train_dg']
ecog_3_valid = raw['test_data'][50000:200000]

dg_1_raw = scipy.io.loadmat('./datasets/sub1_testlabels.mat')
dg_1_valid = dg_1_raw['test_dg'][50000:200000]
dg_1_valid = np.delete(dg_1_valid, 3, 1)

dg_2_raw = scipy.io.loadmat('./datasets/sub2_testlabels.mat')
dg_2_valid = dg_2_raw['test_dg'][50000:200000]
dg_2_valid = np.delete(dg_2_valid, 3, 1)

dg_3_raw = scipy.io.loadmat('./datasets/sub3_testlabels.mat')
dg_3_valid = dg_3_raw['test_dg'][50000:200000]
dg_3_valid = np.delete(dg_3_valid, 3, 1)

In [74]:
dg_2_comp.shape[0] - 49000

351000

In [75]:
winLen = 100 / 1e3
winOverlap = 50 / 1e3
winDisp = winLen - winOverlap
NumWins(ecog_1, 1000, winLen, winDisp)

feature_1 = get_windowed_feats(ecog_1_valid, 1000, winLen, winOverlap)
# y_1 = sig.resample(dg_1_valid, feature_1.shape[0], axis=0)
feature_2 = get_windowed_feats(ecog_2_valid, 1000, winLen, winOverlap)
# y_2 = sig.resample(dg_2_valid, feature_2.shape[0], axis=0)
feature_3 = get_windowed_feats(ecog_3_valid, 1000, winLen, winOverlap)
# y_3 = sig.resample(dg_3_valid, feature_3.shape[0], axis=0)

idx_1 = np.load('./models/idx_S1.npy')
idx_2 = np.load('./models/idx_S2.npy')
idx_3 = np.load('./models/idx_S3.npy')

R_1 = create_R_matrix(feature_1, 20)[:, idx_1]
R_2 = create_R_matrix(feature_2, 20)[:, idx_2]
R_3 = create_R_matrix(feature_3, 20)[:, idx_3]

R_list = [R_1, R_2, R_3]

In [85]:
train_mean_S1 = np.load('./models/train_mean_S1.npy')
train_std_S1 = np.load('./models/train_std_S1.npy')

train_mean_S2 = np.load('./models/train_mean_S2.npy')
train_std_S2 = np.load('./models/train_std_S2.npy')

train_mean_S3 = np.load('./models/train_mean_S3.npy')
train_std_S3 = np.load('./models/train_std_S3.npy')

train_mean_ls = [train_mean_S1, train_mean_S2, train_mean_S3]
train_std_ls = [train_std_S1, train_std_S2, train_std_S3]

In [77]:
device = torch.device("cpu")

In [78]:
import lightgbm
lightgbm.Booster(model_file=f'./models/lgbr_f{0}_S{1}.txt')

In [83]:
predictions = []
for i in range(3):
    # Load XGB
    xgb_reg = xgb.XGBRegressor()
    xgb_reg.load_model(f"./models/XGB_S{i + 1}.json")

    prediction_xgb = xgb_reg.predict(R_list[i])
    
    # Load LGBM
    lgbm_reg_list = [lightgbm.Booster(model_file=f'./models/lgbr_f{j}_S{i + 1}.txt') for j in range(4)]
    
    prediction_lgbm_list = [lgbm_reg.predict(R_list[i]) for lgbm_reg in lgbm_reg_list]
    prediction_lgbm = np.vstack(prediction_lgbm_list).T
    
#     Load NN
    leaderboard_dataset = FingerFeatureDataset((R_list[i] - train_mean_ls[i]) / train_std_ls[i], np.zeros(R_list[i].shape[0]).copy())
    dataloader = DataLoader(leaderboard_dataset, batch_size=16, shuffle=False)

    net = FingerRegressor(R_list[i].shape[1], 4).to(device)
    net.load_state_dict(torch.load(f'./models/NN_S{i + 1}.pth', map_location=device))
    
    with torch.no_grad():
        pred = []
        net.eval()
        for i, (ecog, dg) in enumerate(dataloader):
            ecog = ecog.to(device)
            dg = dg.to(device)
            output = net(ecog).to(device)
            pred += [output.detach().cpu().numpy()]

    prediction_NN = np.concatenate(pred)
    
    
    prediction = (prediction_lgbm + prediction_NN) / 2
    predictions.append(prediction)

ValueError: operands could not be broadcast together with shapes (2999,617) (615,) 

In [81]:
train_mean_ls[i].shape

(615,)

In [31]:
time = np.arange(0, ecog_1_valid.shape[0] / 1000, 1 / 1000)
sample_time = np.linspace(0, ecog_1_valid.shape[0] / 1000, R_1.shape[0])

In [50]:
def post_processing(prediction, time, sample_time):
    f = scipy.interpolate.PchipInterpolator(sample_time, prediction, axis=0)
    
    return f(time)

In [51]:
prediction1 = post_processing(predictions[0], time, sample_time)

In [24]:
pack_submission(predictions, 'validation.mat', 'predicted_dg')

In [25]:
predictions = scipy.io.loadmat('validation.mat')['predicted_dg']
prediction1 = predictions[0][0][:, [0,1,2,4]]
prediction2 = predictions[1][0][:, [0,1,2,4]]
prediction3 = predictions[2][0][:, [0,1,2,4]]

In [52]:
correlation(prediction1, dg_1_valid)

([0.6729982303315438,
  0.7461896842564839,
  0.3407088386454661,
  0.40268559166145235],
 0.5406455862237365)

In [50]:
correlation(prediction2, dg_2_valid)

([0.6805276336790187,
  0.524321604336726,
  0.3798516037347169,
  0.4158781759899006],
 0.5001447544350905)

In [51]:
correlation(prediction3, dg_3_valid)

([0.8092553446980685,
  0.7298080364494511,
  0.6562946518234932,
  0.7558823829933303],
 0.7378101039910858)